In [ ]:
import numpy as np

import scipp as sc
import sciline
from essreflectometry.amor.load import load
from essreflectometry.amor.beamline import make_beamline
from essreflectometry.amor.conversions import specular_reflection
from essreflectometry.amor.resolution import add_resolutions, compute_resolution
from essreflectometry.amor.normalize import normalize_by_supermirror
from essreflectometry.amor.calibrations import supermirror_calibration
from essreflectometry.reflectometry.corrections import footprint_correction, normalize_by_counts
from essreflectometry.reflectometry.conversions import providers
from essreflectometry.reflectometry.types import (
    ThetaBins, WavelengthBins, Sample, Reference, Sample, SampleRotation, Filename,
    ThetaData, WavelengthData, HistogrammedByQ, QDataWithResolutions, QData, QBins, NormalizedIOverQ
)


pipeline = sciline.Pipeline(
    [load, make_beamline, specular_reflection, footprint_correction,
     add_resolutions, compute_resolution, normalize_by_counts, supermirror_calibration, normalize_by_supermirror]
    + providers,
    params={
        ThetaBins: sc.linspace(dim='theta', start=0, stop=np.pi/2, num=2, unit='rad'),
        WavelengthBins: sc.array(dims=['wavelength'], values=[2.4, 16.0], unit='angstrom'),
        QBins: sc.geomspace(dim='Q', start=0.008, stop=0.075, num=200, unit='1/angstrom'),

        SampleRotation[Sample]: sc.scalar(0.7989, unit='deg'),
        Filename[Sample]: "sample.nxs",
        SampleRotation[Reference]: sc.scalar(0.8389, unit='deg'),
        Filename[Reference]: "reference.nxs",
    }
)

pipeline.visualize(NormalizedIOverQ)

In [ ]:
pipeline.compute(HistogrammedByQ[QData[Reference]])

In [ ]:
(pipeline.compute(WithResolution)
     .bins.concat('detector_number').hist(wavelength=200)
     .plot())